In [1]:
# /// script
# requires-python = ">=3.13"
# dependencies = [
#     "numpy",
#     "pyarrow",
# ]
# ///

In [4]:
import numpy as np
import pyarrow as pa
import pyarrow.parquet as pq
from pathlib import Path

keys = [
    "GSM3271347_gm12878_01",
    # "GSM3271347_rep1_gm12878_01",
    # "GSM3271347_rep2_gm12878_01",
    # "GSM3271348_gm12878_02",
    # "GSM3271348_rep1_gm12878_02",
    # "GSM3271348_rep2_gm12878_02",
    # "GSM3271349_gm12878_03",
    # "GSM3271349_rep1_gm12878_03",
    # "GSM3271349_rep2_gm12878_03",
    # "GSM3271350_gm12878_04",
    # "GSM3271350_rep1_gm12878_04",
    # "GSM3271350_rep2_gm12878_04",
    # "GSM3271351_gm12878_05",
    # "GSM3271351_rep1_gm12878_05",
    # "GSM3271351_rep2_gm12878_05",
    # "GSM3271352_gm12878_06",
    # "GSM3271352_rep1_gm12878_06",
    # "GSM3271352_rep2_gm12878_06",
    # "GSM3271353_gm12878_07",
    # "GSM3271353_rep1_gm12878_07",
    # "GSM3271353_rep2_gm12878_07",
    # "GSM3271355_gm12878_09",
    # "GSM3271355_rep1_gm12878_09",
    # "GSM3271355_rep2_gm12878_09",
    # "GSM3271356_gm12878_10",
    # "GSM3271356_rep1_gm12878_10",
    # "GSM3271356_rep2_gm12878_10",
    # "GSM3271358_gm12878_11",
    # "GSM3271358_rep1_gm12878_11",
    # "GSM3271358_rep2_gm12878_11",
    # "GSM3271360_gm12878_12",
    # "GSM3271360_rep1_gm12878_12",
    # "GSM3271360_rep2_gm12878_12",
    # "GSM3271362_gm12878_13",
    # "GSM3271362_rep1_gm12878_13",
    # "GSM3271362_rep2_gm12878_13",
    # "GSM3271364_gm12878_14",
    # "GSM3271364_rep1_gm12878_14",
    # "GSM3271364_rep2_gm12878_14",
    # "GSM3271366_gm12878_15",
    # "GSM3271366_rep1_gm12878_15",
    # "GSM3271366_rep2_gm12878_15",
    # "GSM3271368_gm12878_16",
    # "GSM3271368_rep1_gm12878_16",
    # "GSM3271368_rep2_gm12878_16",
    # "GSM3271370_gm12878_17",
    # "GSM3271370_rep1_gm12878_17",
    # "GSM3271370_rep2_gm12878_17",
    # "GSM3271372_pbmc_01",
    # "GSM3271372_rep1_pbmc_01",
    # "GSM3271372_rep2_pbmc_01",
    # "GSM3271374_pbmc_02",
    # "GSM3271374_rep1_pbmc_02",
    # "GSM3271374_rep2_pbmc_02",
    # "GSM3271376_pbmc_03",
    # "GSM3271376_rep1_pbmc_03",
    # "GSM3271376_rep2_pbmc_03",
    # "GSM3271378_pbmc_04",
    # "GSM3271378_rep1_pbmc_04",
    # "GSM3271378_rep2_pbmc_04",
    # "GSM3271380_pbmc_05",
    # "GSM3271380_rep1_pbmc_05",
    # "GSM3271380_rep2_pbmc_05",
    # "GSM3271382_pbmc_06",
    # "GSM3271382_rep1_pbmc_06",
    # "GSM3271382_rep2_pbmc_06",
    # "GSM3271384_pbmc_07",
    # "GSM3271384_rep1_pbmc_07",
    # "GSM3271384_rep2_pbmc_07",
    # "GSM3271386_pbmc_08",
    # "GSM3271386_rep1_pbmc_08",
    # "GSM3271386_rep2_pbmc_08",
    # "GSM3271388_pbmc_09",
    # "GSM3271388_rep1_pbmc_09",
    # "GSM3271388_rep2_pbmc_09",
    # "GSM3271390_pbmc_10",
    # "GSM3271390_rep1_pbmc_10",
    # "GSM3271390_rep2_pbmc_10",
    # "GSM3271392_pbmc_11",
    # "GSM3271392_rep1_pbmc_11",
    # "GSM3271392_rep2_pbmc_11",
    # "GSM3271394_pbmc_12",
    # "GSM3271394_rep1_pbmc_12",
    # "GSM3271394_rep2_pbmc_12",
    # "GSM3271396_pbmc_13",
    # "GSM3271396_rep1_pbmc_13",
    # "GSM3271396_rep2_pbmc_13",
    # "GSM3271398_pbmc_14",
    # "GSM3271398_rep1_pbmc_14",
    # "GSM3271398_rep2_pbmc_14",
    # "GSM3271400_pbmc_15",
    # "GSM3271400_rep1_pbmc_15",
    # "GSM3271400_rep2_pbmc_15",
    # "GSM3271402_pbmc_16",
    # "GSM3271402_rep1_pbmc_16",
    # "GSM3271402_rep2_pbmc_16",
    # "GSM3271404_pbmc_17",
    # "GSM3271404_rep1_pbmc_17",
    # "GSM3271404_rep2_pbmc_17",
    # "GSM3271406_pbmc_18",
    # "GSM3271406_rep1_pbmc_18",
    # "GSM3271406_rep2_pbmc_18",
]

In [6]:
basepath = "GSE117876_RAW/selected"
postfix = ".impute.3dg.txt"
prefix = "Tan-2018_"

out_folder = Path("out")
out_folder.mkdir(parents=True, exist_ok=True)

def arrays_to_table(x_arr, y_arr, z_arr, chrom_arr, coord_arr):
    x_nparr = np.array(x_arr)
    y_nparr = np.array(y_arr)
    z_nparr = np.array(z_arr)
    chrom_nparr = np.array(chrom_arr)
    coord_nparr = np.array(coord_arr)
    return pa.Table.from_arrays([x_nparr, y_nparr, z_nparr, chrom_nparr, coord_nparr], ["x", "y", "z", "chr", "coord"])

def write_to_file(outfile_path, table):
    with pa.OSFile(outfile_path, 'wb') as sink:
        with pa.ipc.RecordBatchFileWriter(sink, table.schema) as writer:
            writer.write_table(table)

outputs_list_file = "out/out_file.txt"
with open(outputs_list_file, "w") as outputs_file:
    for key in keys:
        filepath = f"{basepath}/{key}{postfix}"
        print("input: " + filepath)
        with open(filepath, 'r') as file:
            # line_count = sum(1 for _ in file)
            # print(line_count)
            x_arr = []
            y_arr = []
            z_arr = []
            chr_arr = []
            coord_arr = []
            for line in file:
                tokens = line.strip().split("\t")
                x_arr.append(float(tokens[2]))
                y_arr.append(float(tokens[3]))
                z_arr.append(float(tokens[4]))
                chr_arr.append(tokens[0])
                coord_arr.append(int(tokens[1]))
                # break
    
            # print(x_arr)
            table = arrays_to_table(x_arr, y_arr, z_arr, chr_arr, coord_arr)
            # print(table)
            
            outputpath_arrow = f'out/{prefix}{key}.arrow'
            print("Writing: " + outputpath_arrow)
            write_to_file(outputpath_arrow, table)
            
            outputs_file.write(f'{prefix}{key}\n')
    
        # break # only do one for now

input: GSE117876_RAW/selected/GSM3271347_gm12878_01.impute.3dg.txt
Writing: out/Tan-2018_GSM3271347_gm12878_01.arrow
